In [ ]:
import os
import logging
from dataclasses import dataclass
import random
import numpy as np
import pandas as pd
import tensorflow as tf
import keras
from sklearn.metrics import confusion_matrix, ConfusionMatrixDisplay
import matplotlib.pyplot as plt
from sklearn.utils.class_weight import compute_class_weight

# import keras_tuner as kt


JESTER_DATASET_DIR_PATH = os.path.join("..", "..", "datasets")
TRAIN_INFO_PATH = os.path.join(JESTER_DATASET_DIR_PATH, "info", "jester-v1-train.csv")
VAL_INFO_PATH = os.path.join(JESTER_DATASET_DIR_PATH, "info", "jester-v1-validation.csv")
TEST_INFO_PATH = os.path.join(JESTER_DATASET_DIR_PATH, "info", "labeled-test.csv")
LABELS_INFO_PATH = os.path.join(JESTER_DATASET_DIR_PATH, "info", 'jester-v1-labels.csv')
VIDEO_DIR_PATH = os.path.join(JESTER_DATASET_DIR_PATH, "data")


train_df = pd.read_csv(os.path.join(JESTER_DATASET_DIR_PATH, "info", "train_df.csv"))
val_df = pd.read_csv(os.path.join(JESTER_DATASET_DIR_PATH, "info", "val_df.csv"))
test_df = pd.read_csv(os.path.join(JESTER_DATASET_DIR_PATH, "info", "test_df.csv"))

columns = ["label"]
labels_info = pd.read_csv(os.path.join(JESTER_DATASET_DIR_PATH, "info", "jester-v1-labels.csv"), names=columns)


In [ ]:
train_df

In [ ]:
val_df

In [ ]:
test_df

In [ ]:
SET_SIZE = (121_500 + len(val_df) + len(test_df) ) // 1
TRAIN_SET_SIZE = 121_500 // 1
VAL_SET_SIZE = len(val_df) // 1
TEST_SET_SIZE = len(test_df) // 1
MIN_SEQ_LEN = 30
CLASSES = [i for i in range(27)]
CLASSES_PERCENTAGE = [1/len(CLASSES) for _ in range(len(CLASSES))]
CLASSES_WITH_PERCENTAGE = {cls: percentage for cls, percentage in zip(CLASSES, CLASSES_PERCENTAGE)}
TRAIN_CLASSES_WITH_AMOUNT = {cls: int(percentage * TRAIN_SET_SIZE) for cls, percentage in zip(CLASSES, CLASSES_PERCENTAGE)}
TRAIN_CLASSES_WITH_AMOUNT[26] = TRAIN_CLASSES_WITH_AMOUNT.get(26) + 4500 // 1
# TRAIN_CLASSES_WITH_AMOUNT[14] = TRAIN_CLASSES_WITH_AMOUNT.get(14) - 1300 // 1
# TRAIN_CLASSES_WITH_AMOUNT[15] = TRAIN_CLASSES_WITH_AMOUNT.get(15) - 1300 // 1
VAL_CLASSES_WITH_AMOUNT = {cls: int(percentage * VAL_SET_SIZE) for cls, percentage in zip(CLASSES, CLASSES_PERCENTAGE)}
VAL_CLASSES_WITH_AMOUNT[26] = VAL_CLASSES_WITH_AMOUNT.get(26) + 450 // 1
TEST_CLASSES_WITH_AMOUNT = {cls: int(percentage * TEST_SET_SIZE) for cls, percentage in zip(CLASSES, CLASSES_PERCENTAGE)}
TEST_CLASSES_WITH_AMOUNT[26] = TEST_CLASSES_WITH_AMOUNT.get(26) + 450 // 1
CLASS_MAPPING = {cls: idx for idx, cls in enumerate(CLASSES)}

In [ ]:
TRAIN_SET_SIZE / (SET_SIZE)

In [ ]:
VAL_CLASSES_WITH_AMOUNT

In [ ]:
TRAIN_CLASSES_WITH_AMOUNT

In [ ]:
TEST_CLASSES_WITH_AMOUNT

In [ ]:
def preprocess_df(dataframe: pd.DataFrame, subset_info: dict[int, int], min_seq_len: int,
                  training: bool) -> pd.DataFrame:
    def _get_longer_seq_len_than(df: pd.DataFrame, seq_len: int) -> pd.DataFrame:
        return df[df["seq_len"] >= seq_len]

    def _get_df_subset(df: pd.DataFrame, classes_with_amount: dict[int, int], training: bool) -> pd.DataFrame:
        subset_df = pd.DataFrame()
        for key, val in classes_with_amount.items():
            class_df = df[df["label_id"] == key]
            current_count = len(class_df)

            if training:
                if current_count < val:
                    needed_count = val - current_count
                    sampled_df = class_df.sample(n=needed_count, replace=True)
                    class_df = pd.concat([class_df, sampled_df], ignore_index=True)
                subset_df = pd.concat([subset_df, class_df.sample(n=val, replace=False)], ignore_index=True)
            else:
                subset_df = pd.concat([subset_df, class_df.sample(n=min(val, current_count), replace=False)],
                                      ignore_index=True)

        return subset_df.sample(frac=1, ignore_index=True)

    # new_df = _get_longer_seq_len_than(dataframe, min_seq_len)
    new_df = _get_df_subset(dataframe, subset_info, training)
    return new_df


train_subset_df = preprocess_df(train_df, TRAIN_CLASSES_WITH_AMOUNT, MIN_SEQ_LEN, training=True)
val_subset_df = preprocess_df(val_df, VAL_CLASSES_WITH_AMOUNT, MIN_SEQ_LEN, training=False)
test_subset_df = preprocess_df(test_df, TEST_CLASSES_WITH_AMOUNT, MIN_SEQ_LEN, training=False)



In [ ]:
labels = train_subset_df['label_id'].map(CLASS_MAPPING).values
labels

In [ ]:
class_weights = compute_class_weight(
    class_weight='balanced',
    classes=np.unique(labels),
    y=labels
)

class_weight_dict = dict(zip(np.unique(labels), class_weights))

In [ ]:
class_weight_dict

In [ ]:
train_subset_df

In [ ]:
val_subset_df

In [ ]:
test_subset_df

In [ ]:
plt.figure(figsize=(16, 9))
train_subset_df['label'].value_counts().plot(kind='bar', color='skyblue', edgecolor='black')
plt.xlabel('Etykiety', fontsize=18)
plt.ylabel('Ilość Próbek', fontsize=18)
plt.title('Rozkład próbek w zbiorze treningowym po przetworzeniu', fontsize=22)
ax = plt.gca()
ax.set_axisbelow(True)
plt.grid(True, which='both', linestyle='--', linewidth=0.5, color='gray')
ax.tick_params(axis='both', which='major', labelsize=16)
plt.xticks(rotation=45, ha='right', fontsize=16)
plt.tight_layout()
plt.savefig("cls_dist_train_after.jpg", dpi=300)
plt.show()

In [ ]:
plt.figure(figsize=(16, 9))
val_subset_df['label'].value_counts().plot(kind='bar', color='skyblue', edgecolor='black')
plt.xlabel('Etykiety', fontsize=18)
plt.ylabel('Ilość Próbek', fontsize=18)
plt.title('Rozkład próbek w zbiorze walidacyjnym po przetworzeniu', fontsize=22)
ax = plt.gca()
ax.set_axisbelow(True)
plt.grid(True, which='both', linestyle='--', linewidth=0.5, color='gray')
ax.tick_params(axis='both', which='major', labelsize=16)
plt.xticks(rotation=45, ha='right', fontsize=16)
plt.tight_layout()
plt.savefig("cls_dist_val_after.jpg", dpi=300)
plt.show()

In [ ]:
plt.figure(figsize=(16, 9))
test_subset_df['label'].value_counts().plot(kind='bar', color='skyblue', edgecolor='black')
plt.xlabel('Etykiety', fontsize=18)
plt.ylabel('Ilość Próbek', fontsize=18)
plt.title('Rozkład próbek w zbiorze testowym po przetworzeniu', fontsize=22)
ax = plt.gca()
ax.set_axisbelow(True)
plt.grid(True, which='both', linestyle='--', linewidth=0.5, color='gray')
ax.tick_params(axis='both', which='major', labelsize=16)
plt.xticks(rotation=45, ha='right', fontsize=16)
plt.tight_layout()
plt.savefig("cls_dist_test_after.jpg", dpi=300)
plt.show()

In [ ]:
import matplotlib.pyplot as plt
all_df = pd.concat([train_subset_df, val_subset_df, test_subset_df])
plt.figure(figsize=(16, 9))
all_df['seq_len'].hist(bins=50, color='skyblue', edgecolor='black')
plt.xlabel('Długość sekwencji', fontsize=18)
plt.ylabel('Liczba wystąpień', fontsize=18)
plt.title('Rozkład długości próbek w zbiorze', fontsize=22)
ax = plt.gca()
ax.set_axisbelow(True)
plt.grid(True, which='both', linestyle='--', linewidth=0.5, color='gray')
ax.tick_params(axis='both', which='major', labelsize=16)
plt.tight_layout()
plt.savefig("jester_plots/seq_len_distribution_after.jpg", dpi=300)
plt.show()

In [ ]:
print(f"Mean Sequence Length In Train Dataset: {train_subset_df['seq_len'].mean()}")
print(f"Mean Sequence Length In Validation Dataset: {val_subset_df['seq_len'].mean()}")

In [ ]:
def create_dataset_from_dir(df: pd.DataFrame, seq_len: int, class_mapping: dict[int, int]):
    sequences = []
    labels = []

    for _, row in df.iterrows():
        video_path = row['path']
        label_id = row['label_id']
        image_paths = generate_image_paths(video_path, seq_len, row)
        sequences.append(image_paths)
        mapped_label_id = class_mapping[label_id]
        labels.append(mapped_label_id)

    dataset = tf.data.Dataset.from_tensor_slices((sequences, labels))
    return dataset


@dataclass
class InputShape:
    weight: int
    height: int
    channels: int

    def as_tuple(self) -> tuple[int, int, int]:
        return self.weight, self.height, self.channels


def generate_image_paths(video_dir_path: str, seq_len: int, row: pd.Series):
    image_paths = []
    current_frames: int = row["seq_len"]

    if current_frames >= seq_len:
        mid_point = current_frames // 2
        start_point = max(0, mid_point - seq_len // 2)
        end_point = start_point + seq_len

        for i in range(start_point + 1, end_point + 1):
            img_name = f"{i:05d}.jpg"
            img_path = os.path.join(video_dir_path, img_name)
            image_paths.append(img_path)

    else:
        padding_needed = seq_len - current_frames
        left_padding = padding_needed // 2
        right_padding = padding_needed - left_padding

        most_left_img = f"00001.jpg"
        most_right_img = f"{current_frames:05d}.jpg"

        for i in range(left_padding):
            img_path = os.path.join(video_dir_path, most_left_img)
            image_paths.append(img_path)

        for i in range(1, current_frames + 1):
            img_name = f"{i:05d}.jpg"
            img_path = os.path.join(video_dir_path, img_name)
            image_paths.append(img_path)

        for i in range(right_padding):
            img_path = os.path.join(video_dir_path, most_right_img)
            image_paths.append(img_path)

    if len(image_paths) != seq_len:
        raise ValueError(f"Missing images in dir: {video_dir_path}")

    return image_paths


@tf.function
def preprocess_frames(frames, label):
    # flip = random.choice([True, False])
    brightness_delta = random.uniform(-0.2, 0.2)
    contrast_factor = random.uniform(0.8, 1.2)
    saturation_factor = random.uniform(0.8, 1.2)
    hue_delta = random.uniform(-0.02, 0.02)
    noise_stddev = 0.05

    augmentations = [
        # lambda img: tf.image.flip_left_right(img) if flip else img,
        lambda img: tf.image.adjust_brightness(img, brightness_delta),
        lambda img: tf.image.adjust_contrast(img, contrast_factor),
        lambda img: tf.image.adjust_saturation(img, saturation_factor),
        lambda img: tf.image.adjust_hue(img, hue_delta),
        lambda img: img + tf.random.normal(tf.shape(img), mean=0.0, stddev=noise_stddev)
    ]

    chosen_augmentations = random.sample(augmentations, 2)

    def process_image(img):
        for aug in chosen_augmentations:
            img = aug(img)
        # img = tf.cast(img, tf.float32) / 255.0
        return img

    preprocessed_images = tf.map_fn(process_image, frames, fn_output_signature=tf.float32)

    return preprocessed_images, label


@tf.function
def load_sequence_from_dir(image_paths: tf.Tensor, label: int, inp_shape: tuple[int, int, int]):
    def process_image(img_path):
        image = tf.io.read_file(img_path)
        image = tf.image.decode_jpeg(image, channels=inp_shape[2])
        image = tf.image.resize(image, inp_shape[:2])
        # image = tf.cast(image, tf.float32) / 255.0
        return image

    preprocessed_images = tf.map_fn(process_image, image_paths, fn_output_signature=tf.float32)

    return preprocessed_images, label


@tf.function
def one_hot_encode(path: tf.Tensor, label: tf.Tensor, classes_num: int):
    return path, tf.one_hot(label, classes_num, dtype=tf.int32)


@tf.function
def remove_one_dimensions(images: tf.Tensor, label: int):
    return tf.squeeze(images), label

@tf.function
def normalize_frames(frames, label):
    return (tf.cast(frames, tf.float32) / 255.0), label

@tf.function
def add_dimension(frames, label):
    frames = tf.expand_dims(frames, axis=0) 
    return frames, label

def create_pipeline(df: pd.DataFrame, *, num_classes: int, image_input_shape: InputShape, seq_len: int, batch_size: int, class_mapping: dict[int, int], is_training: bool = False) -> tf.data.Dataset:
    num_calls = tf.data.AUTOTUNE
    ds = create_dataset_from_dir(df, seq_len=seq_len, class_mapping=class_mapping) # (list with paths strs)
    ds = ds.map(lambda images, label: one_hot_encode(images, label, num_classes), num_parallel_calls=num_calls)
    ds = ds.map(lambda path, label: load_sequence_from_dir(path, label, image_input_shape.as_tuple()), num_parallel_calls=num_calls) # (seq_len, width, height, channels)
    if is_training:
        pass
        ds = ds.map(lambda frames, label: preprocess_frames(frames, label), num_parallel_calls=num_calls)
    ds = ds.map(lambda frames, label: normalize_frames(frames, label), num_parallel_calls=num_calls)
    # ds = ds.map(lambda frames, label: add_dimension(frames, label), num_parallel_calls=num_calls)
    # ds = ds.map(lambda frames, label: preprocess_first_layer(frames, label), num_parallel_calls=num_calls)
    # ds = ds.map(lambda frames, label: remove_one_dimensions(frames, label), num_parallel_calls=num_calls)
    ds = ds.batch(batch_size=batch_size, drop_remainder=True)
    # if is_training:
    #     ds = ds.cache("./cache_mobilenet_train")
    # else:
    #     ds = ds.cache("./cache_mobilenet_val")
    ds = ds.prefetch(num_calls)
    return ds


class PipelineConfig:
    IMAGE_INPUT_SHAPE = InputShape(224, 224, 3)
    NUM_CLASSES = len(CLASSES)
    BATCH_SIZE = 16
    SEQ_LEN = 34


# mobilenet_model = keras.applications.MobileNetV2(
#     include_top=False,
#     weights="imagenet",
#     input_shape=PipelineConfig.IMAGE_INPUT_SHAPE.as_tuple(),
#     pooling="avg",
#     name="mobilenet_v2"
# )

# time_distributed_layer = keras.layers.TimeDistributed(mobilenet_model)

# def preprocess_first_layer(frames, label):
#     first_layer_output = time_distributed_layer(frames)
#     return first_layer_output, label



train_ds = create_pipeline(
    train_subset_df,
    num_classes=PipelineConfig.NUM_CLASSES,
    image_input_shape=PipelineConfig.IMAGE_INPUT_SHAPE,
    seq_len=PipelineConfig.SEQ_LEN,
    batch_size=PipelineConfig.BATCH_SIZE,
    class_mapping=CLASS_MAPPING,
    is_training=True
)

val_ds = create_pipeline(
    val_subset_df,
    num_classes=PipelineConfig.NUM_CLASSES,
    image_input_shape=PipelineConfig.IMAGE_INPUT_SHAPE,
    seq_len=PipelineConfig.SEQ_LEN,
    batch_size=PipelineConfig.BATCH_SIZE,
    class_mapping=CLASS_MAPPING,
    is_training=False
)

test_ds = create_pipeline(
    test_subset_df,
    num_classes=PipelineConfig.NUM_CLASSES,
    image_input_shape=PipelineConfig.IMAGE_INPUT_SHAPE,
    seq_len=PipelineConfig.SEQ_LEN,
    batch_size=PipelineConfig.BATCH_SIZE,
    class_mapping=CLASS_MAPPING,
    is_training=False
)

# Articles Based Model

In [ ]:
def build_model_from_articles(input_shape: tuple[int, int], num_classes: int):
    mobilenet_model = keras.applications.MobileNetV3Large(
        include_top=False,
        weights="imagenet",
        input_shape=PipelineConfig.IMAGE_INPUT_SHAPE.as_tuple(),
        include_preprocessing=True,
        pooling="avg",
    )
    model = keras.models.Sequential()
    model.add(keras.layers.Input(input_shape))
    model.add(keras.layers.TimeDistributed(mobilenet_model))
    for layer in model.layers:
        layer.trainable = False

    model.add(keras.layers.LSTM(units=256, return_sequences=True, kernel_regularizer=keras.regularizers.l2(0.0001), name="LSTM_1"))
    model.add(keras.layers.Dropout(0.2, name="LSTM_DROPOUT_1"))
    model.add(keras.layers.LayerNormalization(name="LSTM_LNORM_1"))
    model.add(keras.layers.LSTM(units=256, return_sequences=True, kernel_regularizer=keras.regularizers.l2(0.0001), name="LSTM_2"))
    model.add(keras.layers.Dropout(0.2, name="LSTM_DROPOUT_2"))
    model.add(keras.layers.LayerNormalization(name="LSTM_LNORM_2"))
    model.add(keras.layers.LSTM(units=256, return_sequences=False, kernel_regularizer=keras.regularizers.l2(0.0001), name="LSTM_3"))
    model.add(keras.layers.Dropout(0.2, name="LSTM_DROPOUT_3"))
    model.add(keras.layers.LayerNormalization(name="LSTM_LNORM_3"))
    model.add(keras.layers.Dense(units=256, activation="tanh", kernel_regularizer=keras.regularizers.l2(0.001), name="DENSE_1"))
    model.add(keras.layers.Dropout(0.1, name="DENSE_DROPOUT_1"))
    model.add(keras.layers.LayerNormalization(name="DENSE_LNORM_1"))
    # model.add(keras.layers.Dense(units=256, activation="tanh", kernel_regularizer=keras.regularizers.l2(0.0001), name="DENSE_2"))
    # model.add(keras.layers.Dropout(0.1, name="DENSE_DROPOUT_2"))
    # model.add(keras.layers.LayerNormalization(name="DENSE_LNORM_2"))
    model.add(keras.layers.Dense(units=num_classes, name="DENSE_OUTPUT"))

    optimizer = keras.optimizers.Adam(0.001)

    model.compile(
    optimizer=optimizer,
    loss=keras.losses.CategoricalCrossentropy(from_logits=True),
        metrics=[
        'accuracy',
        'precision',
        'recall',
        tf.keras.metrics.F1Score(average='weighted')
        ]
    )
    return model

In [ ]:
lstm_model = build_model_from_articles(input_shape=(PipelineConfig.SEQ_LEN, *PipelineConfig.IMAGE_INPUT_SHAPE.as_tuple()), num_classes=len(CLASSES))

early_stopping_callback = keras.callbacks.EarlyStopping(
    patience=10,
    monitor='val_loss',
    mode="min"
)

checkpoint_callback = keras.callbacks.ModelCheckpoint(
    'model_articles_based.keras',
    save_best_only=True,
    monitor='val_loss',
    mode='min',
    verbose=1
)

lstm_model.summary()

# history = lstm_model.fit(
#     train_ds, epochs=100,
#     validation_data=val_ds,
#     batch_size=PipelineConfig.BATCH_SIZE,
#     callbacks=[early_stopping_callback, checkpoint_callback],
#     class_weight=class_weight_dict
# )



In [ ]:
def build_model_from_articles_conv3D(input_shape: tuple[int, int, int, int], num_classes: int):
    model = keras.models.Sequential()
    model.add(keras.layers.Input(input_shape))

    # Warstwa 1 - conv3D
    model.add(keras.layers.Conv3D(32, kernel_size=(3, 3, 3), strides=(1, 1, 1), kernel_regularizer=keras.regularizers.l2(0.0001), activation='relu', padding='same'))
    model.add(keras.layers.BatchNormalization())

    # Warstwa 2 - max pool
    model.add(keras.layers.MaxPooling3D(pool_size=(1, 2, 2)))
#    model.add(keras.layers.Dropout(0.1))

    # Warstwa 3 - conv3D
    model.add(keras.layers.Conv3D(64, kernel_size=(3, 3, 3), strides=(1, 1, 1), kernel_regularizer=keras.regularizers.l2(0.0001), activation='relu', padding='same'))
    model.add(keras.layers.BatchNormalization())

    # Warstwa 4 - max pool
    model.add(keras.layers.MaxPooling3D(pool_size=(1, 2, 2)))
 #   model.add(keras.layers.Dropout(0.1))
    # Warstwa 5 - conv3D
    model.add(keras.layers.Conv3D(128, kernel_size=(3, 3, 3), strides=(1, 1, 1), kernel_regularizer=keras.regularizers.l2(0.0001), activation='relu', padding='same'))
    model.add(keras.layers.BatchNormalization())

    # Warstwa 6 - max pool
    model.add(keras.layers.MaxPooling3D(pool_size=(1, 2, 2)))
 #   model.add(keras.layers.Dropout(0.1))
    # Warstwa 7 - conv3D
    model.add(keras.layers.Conv3D(256, kernel_size=(3, 3, 3), strides=(1, 1, 1), kernel_regularizer=keras.regularizers.l2(0.0001), activation='relu', padding='same'))
    model.add(keras.layers.BatchNormalization())
 #   model.add(keras.layers.Dropout(0.1))
    # Warstwa 8 - conv3D
    model.add(keras.layers.Conv3D(256, kernel_size=(3, 3, 3), strides=(1, 1, 1), kernel_regularizer=keras.regularizers.l2(0.0001), activation='relu', padding='same'))
    model.add(keras.layers.BatchNormalization())
 #   model.add(keras.layers.Dropout(0.1))
    # Warstwa 9 - conv3D
    model.add(keras.layers.Conv3D(256, kernel_size=(3, 3, 3), strides=(1, 1, 1), kernel_regularizer=keras.regularizers.l2(0.0001), activation='relu', padding='same'))
    model.add(keras.layers.BatchNormalization())
 #   model.add(keras.layers.Dropout(0.1))    
    # Warstwa 10 - global max pool
    model.add(keras.layers.GlobalMaxPooling3D())
    # Warstwa 11 - LSTM
    model.add(keras.layers.RepeatVector(1))
    model.add(keras.layers.LSTM(256, kernel_regularizer=keras.regularizers.l2(0.0001), return_sequences=True))
 #   model.add(keras.layers.Dropout(0.1))
    # Warstwa 12 - LSTM
    model.add(keras.layers.LSTM(256, kernel_regularizer=keras.regularizers.l2(0.0001)))
    model.add(keras.layers.LayerNormalization(name="DENSE_LNORM_1"))
 #   model.add(keras.layers.Dropout(0.1))
    # Warstwa 13 - Fully connected
    model.add(keras.layers.Dense(units=256, activation="relu", kernel_regularizer=keras.regularizers.l2(0.0001), name="DENSE_LAYER"))
    model.add(keras.layers.LayerNormalization(name="DENSE_LNORM_2"))
 #   model.add(keras.layers.Dropout(0.1))
    # Warstwa 14 - Fully connected (output)
    model.add(keras.layers.Dense(units=num_classes, name="DENSE_OUTPUT"))

    # Optimizer and compile
    optimizer = keras.optimizers.Adam(0.001)
    model.compile(
        optimizer=optimizer,
        loss=keras.losses.CategoricalCrossentropy(from_logits=True),
        metrics=[
            'accuracy',
            'precision',
            'recall',
            tf.keras.metrics.F1Score(average='weighted')
        ]
    )

    return model

# Plots From Articles

### Grid

In [ ]:
plt.figure(figsize=(15, 7))

plt.subplot(1, 2, 1)
plt.plot(history.history['accuracy'], label='Dokładność na zbiorze treningowym')
plt.plot(history.history['val_accuracy'], label='Dokładność na zbiorze walidacyjnym')
plt.xlabel('Epoka')
plt.ylabel('Dokładność')
plt.ylim([0, 1])
plt.legend(loc='lower right')
plt.grid()
plt.title('Krzywa Uczenia - Dokładność')

# Loss plot
plt.subplot(1, 2, 2)
plt.plot(history.history['loss'], label='Strata na zbiorze treningowym')
plt.plot(history.history['val_loss'], label='Strata na zbiorze walidacyjnym')
plt.xlabel('Epoka')
plt.ylabel('Strata')
plt.ylim([0, max(history.history['loss'])])
plt.legend(loc='upper right')
plt.grid()
plt.title('Krzywa Uczenia - Strata')


plt.savefig('training_plots_articles_based_loss_acc.png')

print("Plots saved as 'training_plots.png'")

In [ ]:
plt.figure(figsize=(15, 25))

plt.subplot(3, 2, 1)
plt.plot(history.history['accuracy'], label='Dokładność na zbiorze treningowym')
plt.plot(history.history['val_accuracy'], label='Dokładność na zbiorze walidacyjnym')
plt.xlabel('Epoka')
plt.ylabel('Dokładność')
plt.ylim([0, 1])
plt.legend(loc='lower right')
plt.grid()
plt.title('Krzywa Uczenia - Dokładność')

plt.subplot(3, 2, 2)
plt.plot(history.history['loss'], label='Strata na zbiorze treningowym')
plt.plot(history.history['val_loss'], label='Strata na zbiorze walidacyjnym')
plt.xlabel('Epoka')
plt.ylabel('Strata')
plt.ylim([0, max(history.history['loss'])])
plt.legend(loc='upper right')
plt.grid()
plt.title('Krzywa Uczenia - Strata')

plt.subplot(3, 2, 3)
plt.plot(history.history['precision'], label='Precyzja na zbiorze treningowym')
plt.plot(history.history['val_precision'], label='Precyzja na zbiorze walidacyjnym')
plt.xlabel('Epoka')
plt.ylabel('Precyzja')
plt.ylim([0, 1])
plt.legend(loc='lower right')
plt.grid()
plt.title('Krzywa Uczenia - Precyzja')

plt.subplot(3, 2, 4)
plt.plot(history.history['recall'], label='Czułość na zbiorze treningowym')
plt.plot(history.history['val_recall'], label='Czułość na zbiorze walidacyjnym')
plt.xlabel('Epoka')
plt.ylabel('Czułość')
plt.ylim([0, 1])
plt.legend(loc='lower right')
plt.grid()
plt.title('Krzywa Uczenia - Czułość')

plt.subplot(3, 2, 5)
plt.plot(history.history['f1_score'], label='F1 Score na zbiorze treningowym')
plt.plot(history.history['val_f1_score'], label='F1 Score na zbiorze walidacyjnym')
plt.xlabel('Epoka')
plt.ylabel('F1 Score')
plt.ylim([0, 1])
plt.legend(loc='lower right')
plt.grid()
plt.title('Krzywa Uczenia - F1 Score')

plt.tight_layout()
plt.savefig('training_plots_articles_based_grid.png')

print("Plots saved as 'training_plots_articles_based.png'")


### No Grid

In [ ]:
plt.figure(figsize=(15, 25))

plt.subplot(3, 2, 1)
plt.plot(history.history['accuracy'], label='Dokładność na zbiorze treningowym')
plt.plot(history.history['val_accuracy'], label='Dokładność na zbiorze walidacyjnym')
plt.xlabel('Epoka')
plt.ylabel('Dokładność')
plt.ylim([0, 1])
plt.legend(loc='lower right')
plt.title('Krzywa Uczenia - Dokładność')

plt.subplot(3, 2, 2)
plt.plot(history.history['loss'], label='Strata na zbiorze treningowym')
plt.plot(history.history['val_loss'], label='Strata na zbiorze walidacyjnym')
plt.xlabel('Epoka')
plt.ylabel('Strata')
plt.ylim([0, max(history.history['loss'])])
plt.legend(loc='upper right')
plt.title('Krzywa Uczenia - Strata')

plt.subplot(3, 2, 3)
plt.plot(history.history['precision'], label='Precyzja na zbiorze treningowym')
plt.plot(history.history['val_precision'], label='Precyzja na zbiorze walidacyjnym')
plt.xlabel('Epoka')
plt.ylabel('Precyzja')
plt.ylim([0, 1])
plt.legend(loc='lower right')
plt.title('Krzywa Uczenia - Precyzja')

plt.subplot(3, 2, 4)
plt.plot(history.history['recall'], label='Czułość na zbiorze treningowym')
plt.plot(history.history['val_recall'], label='Czułość na zbiorze walidacyjnym')
plt.xlabel('Epoka')
plt.ylabel('Czułość')
plt.ylim([0, 1])
plt.legend(loc='lower right')
plt.title('Krzywa Uczenia - Czułość')

plt.subplot(3, 2, 5)
plt.plot(history.history['f1_score'], label='F1 Score na zbiorze treningowym')
plt.plot(history.history['val_f1_score'], label='F1 Score na zbiorze walidacyjnym')
plt.xlabel('Epoka')
plt.ylabel('F1 Score')
plt.ylim([0, 1])
plt.legend(loc='lower right')
plt.title('Krzywa Uczenia - F1 Score')

plt.tight_layout()
plt.savefig('training_plots_articles_based_no_grid.png')

print("Plots saved as 'training_plots_articles_based.png'")


### By one

In [ ]:
plt.figure(figsize=(15, 15))
plt.plot(history.history['accuracy'], label='Dokładność na zbiorze treningowym')
plt.plot(history.history['val_accuracy'], label='Dokładność na zbiorze walidacyjnym')
plt.xlabel('Epoka')
plt.ylabel('Dokładność')
plt.ylim([0, 1])
plt.legend(loc='lower right')
plt.title('Krzywa Uczenia - Dokładność')
plt.grid()
plt.savefig('training_plots_articles_based_acc_grid.png')

print("Plots saved as 'training_plots_articles_based.png'")
plt.figure(figsize=(15, 15))
plt.plot(history.history['loss'], label='Strata na zbiorze treningowym')
plt.plot(history.history['val_loss'], label='Strata na zbiorze walidacyjnym')
plt.xlabel('Epoka')
plt.ylabel('Strata')
plt.ylim([0, max(history.history['loss'])])
plt.legend(loc='upper right')
plt.title('Krzywa Uczenia - Strata')
plt.grid()
plt.savefig('training_plots_articles_based_loss.png')

print("Plots saved as 'training_plots_articles_based.png'")
plt.figure(figsize=(15, 15))
plt.plot(history.history['precision'], label='Precyzja na zbiorze treningowym')
plt.plot(history.history['val_precision'], label='Precyzja na zbiorze walidacyjnym')
plt.xlabel('Epoka')
plt.ylabel('Precyzja')
plt.ylim([0, 1])
plt.legend(loc='lower right')
plt.title('Krzywa Uczenia - Precyzja')
plt.grid()
plt.savefig('training_plots_articles_based_prec.png')

print("Plots saved as 'training_plots_articles_based.png'")
plt.figure(figsize=(15, 15))
plt.plot(history.history['recall'], label='Czułość na zbiorze treningowym')
plt.plot(history.history['val_recall'], label='Czułość na zbiorze walidacyjnym')
plt.xlabel('Epoka')
plt.ylabel('Czułość')
plt.ylim([0, 1])
plt.legend(loc='lower right')
plt.title('Krzywa Uczenia - Czułość')
plt.grid()
plt.savefig('training_plots_articles_based_recall.png')

print("Plots saved as 'training_plots_articles_based.png'")
plt.figure(figsize=(15, 15))
plt.plot(history.history['f1_score'], label='F1 Score na zbiorze treningowym')
plt.plot(history.history['val_f1_score'], label='F1 Score na zbiorze walidacyjnym')
plt.xlabel('Epoka')
plt.ylabel('F1 Score')
plt.ylim([0, 1])
plt.legend(loc='lower right')
plt.title('Krzywa Uczenia - F1 Score')
plt.grid()
plt.savefig('training_plots_articles_based_f1.png')

print("Plots saved as 'training_plots_articles_based.png'")


In [ ]:
for el, label in test_ds:
    print(el.shape)

In [ ]:
class PipelineConfig:
    IMAGE_INPUT_SHAPE = InputShape(164, 164, 3)
    NUM_CLASSES = len(CLASSES)
    BATCH_SIZE = 16
    SEQ_LEN = 34

lstm_model = build_model_from_articles_conv3D(
    input_shape=(PipelineConfig.SEQ_LEN, *PipelineConfig.IMAGE_INPUT_SHAPE.as_tuple()),
    num_classes=PipelineConfig.NUM_CLASSES
)


lstm_model.load_weights('model_weights-3dcnn.weights.h5')

results = lstm_model.evaluate(test_ds)

print("Test Loss:", results[0])
print("Test Accuracy:", results[1])
print("Test Precision:", results[2])
print("Test Recall:", results[3])
print("Test F1 Score:", results[4])

In [ ]:
y_pred = []
y_true = []

for images, labels in test_ds:
    predictions = lstm_model.predict(images)
    predicted_classes = np.argmax(predictions, axis=1)
    
    true_classes = np.argmax(labels, axis=1) if labels.shape[-1] > 1 else labels.numpy()

    y_pred.extend(predicted_classes)
    y_true.extend(true_classes)

cm = confusion_matrix(y_true, y_pred, labels=range(len(CLASSES)))
# cm = confusion_matrix(y_true, y_pred, normalize='true', labels=range(len(CLASSES)))

# print("Confusion Matrix:\n", cm)
# for images, labels in val_ds:
#     predictions = lstm_model.predict(images)
#     predicted_classes = np.argmax(predictions, axis=1)
#     true_classes = np.argmax(labels, axis=1)

#     y_pred.extend(predicted_classes)
#     y_true.extend(true_classes)

# # cm = confusion_matrix(y_true, y_pred, normalize='true', labels=range(len(CLASSES)))
# cm = confusion_matrix(y_true, y_pred, labels=range(len(CLASSES)))

In [ ]:
import matplotlib.pyplot as plt
from sklearn.metrics import ConfusionMatrixDisplay
jester_labels = {}
with open(os.path.join(JESTER_DATASET_DIR_PATH, "info", 'jester-v1-labels.csv')) as f:
    for idx, line in enumerate(f):
        jester_labels[idx] = line.strip()

display_info = [jester_labels.get(cls) for cls in CLASSES]

disp = ConfusionMatrixDisplay(confusion_matrix=cm, display_labels=display_info)

disp.plot(
    cmap=plt.cm.Blues,
    xticks_rotation=45,  
    # values_format='.2f'
    values_format='.0f'
)

fig = disp.figure_
fig.set_figwidth(30)
fig.set_figheight(30)

plt.xlabel('Przewidziana Etykieta', fontsize=20)
plt.ylabel('Prawdziwa Etykieta', fontsize=20)

plt.xticks(fontsize=15, rotation=45, ha='right')
plt.yticks(fontsize=15, rotation=0, va="center")

for text in disp.text_.ravel():
    text.set_fontsize(14)

plt.savefig('conf_mat_BEST_test.png')
print("Plots saved as 'conf_mat_normalized_articles_based_no_norm.png'")
